# LSTM Stock Predictor Using Fear and Greed Index

In this notebook, you will build and train a custom LSTM RNN that uses a 10 day window of Bitcoin fear and greed index values to predict the 11th day closing price. 

You will need to:

1. Prepare the data for training and testing
2. Build and train a custom LSTM RNN
3. Evaluate the performance of the model

## Data Preparation

In this section, you will need to prepare the training and testing data for the model. The model will use a rolling 10 day window to predict the 11th day closing price.

You will need to:
1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the X and y values
4. Reshape the X_train and X_test data for the model. Note: The required input format for the LSTM is:

```python
reshape((X_train.shape[0], X_train.shape[1], 1))
```

In [ ]:
!pip install hvplot

In [7]:
import numpy as np
import pandas as pd
import hvplot.pandas
import io

In [5]:
# Set the random seed for reproducibility
# Note: This is for the homework solution, but it is good practice to comment this out and run multiple experiments to evaluate your model
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [9]:
from google.colab import files
uploaded = files.upload()

Saving btc_historic.csv to btc_historic.csv
Saving btc_sentiment.csv to btc_sentiment.csv


In [10]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv(io.BytesIO(uploaded['btc_sentiment.csv']))
df['Date']= pd.to_datetime(df['date'])
df = df.set_index('Date')
df = df.sort_index()
df = df.drop(['date',"fng_classification"], axis=1)
df.tail()

df.head(2)


,fng_value
Date,
2018-01-02,30
2018-01-03,38


In [11]:
# Load the historical closing prices for Bitcoin
df2 = pd.read_csv(io.BytesIO(uploaded['btc_historic.csv']))
df2['Date']= pd.to_datetime(df2['Date'])
df2 = df2.set_index('Date')
df2 = df2.sort_index()
df2 = df2['Close']
# # df2 = df2.drop(['date'], axis=1)
df2.tail()


Date
2019-07-25    9882.429688
2019-07-26    9847.450195
2019-07-27    9478.320313
2019-07-28    9531.769531
2019-07-29    9529.889648
Name: Close, dtype: float64

In [12]:
# Join the data into a single DataFrame
df = df.join(df2, how="inner")
df.tail()

,fng_value,Close
Date,,
2019-07-25,42,9882.429688
2019-07-26,24,9847.450195
2019-07-27,47,9478.320313
2019-07-28,16,9531.769531
2019-07-29,19,9529.889648


In [13]:
df.head()

,fng_value,Close
Date,,
2018-01-02,30,14754.129883
2018-01-03,38,15156.620117
2018-01-04,16,15180.080078
2018-01-05,56,16954.779297
2018-01-06,24,17172.300781


In [14]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [15]:
# Predict Closing Prices using a 10 day window of previous fng values
# Then, experiment with window sizes anywhere from 1 to 10 and see how the model performance changes
window_size = 10

# Column index 0 is the 'fng_value' column
# Column index 1 is the `Close` column
feature_column = 0
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)

In [16]:
# Use 70% of the data for training and the remaineder for testing
split = int(.7 * len(X))
X_train = X[:split - 1]
X_test = X[split:]
y_train = y[:split - 1]
y_test = y[split:]

In [17]:
from sklearn.preprocessing import MinMaxScaler
# Use the MinMaxScaler to scale data between 0 and 1.
from sklearn.preprocessing import MinMaxScaler
x_train_scaler = MinMaxScaler()
x_test_scaler = MinMaxScaler()
y_train_scaler = MinMaxScaler()
y_test_scaler = MinMaxScaler()

# Fit the scaler for the training Data
x_train_scaler.fit(X_train)
y_train_scaler.fit(y_train)

# Scale the training data
X_train = x_train_scaler.transform(X_train)
y_train = y_train_scaler.transform(y_train)

# Fit the scaler for the testing Data
x_test_scaler.fit(X_test)
y_test_scaler.fit(y_test)

# Scale the y_test data
X_test = x_test_scaler.transform(X_test)
y_test = y_test_scaler.transform(y_test)


In [18]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

---

## Build and Train the LSTM RNN

In this section, you will design a custom LSTM RNN and fit (train) it using the training data.

You will need to:
1. Define the model architecture
2. Compile the model
3. Fit the model to the training data

### Hints:
You will want to use the same model architecture and random seed for both notebooks. This is necessary to accurately compare the performance of the FNG model vs the closing price model. 

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [20]:
# Build the LSTM model. 
# The return sequences need to be set to True if you are adding additional LSTM layers, but 
# You don't have to do this for the final layer. 
# Note: The dropouts help prevent overfitting
# Note: The input shape is the number of time steps and the number of indicators
# Note: Batching inputs has a different input shape of Samples/TimeSteps/Features

model = Sequential()
model.add(LSTM(
    units=30, return_sequences=True,
    input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=30, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=30))
model.add(Dropout(0.2))
model.add(Dense(1))

In [21]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [22]:
# Summarize the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 30)            3840      
_________________________________________________________________
dropout (Dropout)            (None, 10, 30)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 30)            7320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 30)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 30)                7320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 3

In [23]:
# Train the model
# Use at least 10 epochs
# Do not shuffle the data
# Experiement with the batch size, but a smaller batch size is recommended
model.fit(X_train, y_train, epochs=10, shuffle=False, batch_size=1, verbose=1)

Epoch 1/10
346/346 [==============================] - 5s 7ms/step - loss: 0.0202
Epoch 2/10
346/346 [==============================] - 2s 6ms/step - loss: 0.0172
Epoch 3/10
346/346 [==============================] - 2s 7ms/step - loss: 0.0210
Epoch 4/10
346/346 [==============================] - 2s 7ms/step - loss: 0.0221
Epoch 5/10
346/346 [==============================] - 2s 7ms/step - loss: 0.0251
Epoch 6/10
346/346 [==============================] - 2s 7ms/step - loss: 0.0247
Epoch 7/10
346/346 [==============================] - 2s 6ms/step - loss: 0.0257
Epoch 8/10
346/346 [==============================] - 2s 6ms/step - loss: 0.0236
Epoch 9/10
346/346 [==============================] - 2s 7ms/step - loss: 0.0231
Epoch 10/10
346/346 [==============================] - 2s 7ms/step - loss: 0.0220


---

## Model Performance

In this section, you will evaluate the model using the test data. 

You will need to:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart

### Hints
Remember to apply the `inverse_transform` function to the predicted and y_test values to recover the actual closing prices.

In [24]:
# Evaluate the model
model.evaluate(X_test, y_test)

5/5 [==============================] - 1s 5ms/step - loss: 0.1244


0.12439154088497162

In [25]:
# Make some predictions
predicted = model.predict(X_test)

In [26]:
# Recover the original prices instead of the scaled version
predicted_prices = y_test_scaler.inverse_transform(predicted)
real_prices = y_test_scaler.inverse_transform(y_test.reshape(-1, 1))

In [27]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = df.index[-len(real_prices): ]) 
stocks.head()

,Real,Predicted
Date,,
2019-02-01,3434.129883,3483.575439
2019-02-02,3461.629883,3437.510254
2019-02-03,3508.679932,3389.188232
2019-02-04,3449.620117,3338.254150
2019-02-05,3431.239990,3322.713867


In [28]:
# Plot the real vs predicted values as a line chart
stocks.head(100).hvplot()

:NdOverlay   [Variable]
   :Curve   [Date]   (value)